In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.request

import time
import pandas as pd
import os.path
from os import path

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re

#User_agent
from fake_useragent import UserAgent

#implements webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select


#my Imports
from Employee import *
from Tools import *

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (X11; NetBSD) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'}
header

In [ ]:
mainOrganisation = "ApoFinanz"
mainRootUrl = "insert-url"
    
exportDirectory = "Exports/"+ mainOrganisation + "/"
pictureDirectory = "Exports/"+ mainOrganisation + "/Mitarbeiter Bilder/"

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('headless')
chrome_options.add_argument('window-size=1920x1080')
driver = webdriver.Chrome(options=chrome_options)
driver.get(mainRootUrl)

In [ ]:
counter = 0

for person in driver.find_elements_by_xpath("//article[contains(@class, 'contactperson-detail-item') and contains(@class, 'apo-xs-col-6') and contains(@class, 'apo-md-col-2')]"):
    
    if counter == 0:
        counter += 1
        continue
    
    # name
    name_raw = person.find_element_by_xpath(".//span[contains(@class, 'name')]").text
    name_full_list = name_raw.split(" ")
    try:
        if len(name_full_list) > 2 and name_full_list[2] != "":
            name = ' '.join(name_full_list[:len(name_full_list) - 1])
            surname = name_full_list[-1:]
        else:
            name = name_full_list[0]
            surname = name_full_list[1]
    except:
        name = ""
        surname = ""
    
    
    # position & qualification
    #try:
    position_raw = person.text
    position_list = position_raw.split("\n")
    #print(position_raw)
    #print(len(position_list))
    position = position_list[1]
    qualification = position_list[2]
    # print(position)
    # print(qualification)

    
    # phonenumber
    mobile_phonenumber_raw = person.find_element_by_xpath(".//a[contains(@href, 'Mobil')]").text
    mobile_phonenumber = mobile_phonenumber_raw.replace("Mobil: ","").replace(" ","").replace("/","").replace("-","")
    try:
        telefon_phonenumber_raw = person.find_element_by_xpath(".//a[contains(@href, 'Telefon')]").text
        telefon_phonenumber = telefon_phonenumber_raw.replace("Telefon: ","").replace(" ","").replace("/","").replace("-","")
    except:
        telefon_phonenumber = ""

    if "0" in mobile_phonenumber:
        phonenumber = mobile_phonenumber
    else:
        phonenumber = telefon_phonenumber
    print(phonenumber)
    
    
    # image
    image = person.find_element_by_xpath(".//div/img").get_attribute("src")
    # print(image)
    
    
    # address
    address_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-location') and contains(@class, 'js-contactperson-location')]/article[contains(@class, 'contactperson-overview-item')]/p").text
    address_list = address_raw.split("\n")
    street = address_list[0]
    city_list_raw = address_list[1].split(" ")
    plz = city_list_raw[0]
    city = " ".join(city_list_raw[1:])
    # print(street)
    # print(plz)
    # print(city)
    
    
    Employee(surname=surname, name=name, image=image, phonenumber=phonenumber, position=position, city=city, plz=plz, street=street, organisation=organisation, qualification=qualification).add_employee()
    
    counter += 1

In [ ]:
Employee.get_employees()

In [ ]:

    
#----------------------------------------------------------------------------------------------------------------
#Write Employee Excel


Tool = Tools()
cols = ["Funktion","Nachname","Vorname","PLZ (Geschäft)","Ort (Geschäft)","Telefon (Geschäft)","Organisation","Qualifikation", "Bild","Strasse", "Eigene Website"]

dfRaw = pd.DataFrame.from_dict(Employee.get_employees())

df = pd.DataFrame.from_dict(dfRaw)
df = df[cols]

#Write Dataframe to Excel Sheet
writer = pd.ExcelWriter("Exports/"+ mainOrganisation + "/" + mainOrganisation + " Mitarbeiter.xlsx")
df.to_excel(writer, mainOrganisation, index=False)
writer.save()
#----------------------------------------------------------------------------------------------------------------

In [ ]:
crawlerName = mainOrganisation
df = pd.read_excel("Exports/"+ crawlerName + "/" + crawlerName + " Mitarbeiter.xlsx")
#----------------------------------------------------------------------------------------------------------------
#Picture download
Tool = Tools()
downloadError = Tool.employee_picture_download(crawlerName)
print(downloadError)

In [ ]:
class chromeDriver():
    
    # Class Variables:
    
    mainOrganisation = "ApoFinanz"
    mainRootUrl = "https://www.apofinanz.de/berater"
    
    exportDirectory = "Exports/"+ mainOrganisation + "/"
    pictureDirectory = "Exports/"+ mainOrganisation + "/Mitarbeiter Bilder/"
    
    def __init__(self):
        
        chrome_options = webdriver.ChromeOptions()
        #chrome_options.add_argument('headless')
        chrome_options.add_argument('window-size=1920x1080')
        self.browser = webdriver.Chrome(options=chrome_options)
        driver = self.browser
        driver.get(self.mainRootUrl)
        
    
    def get_html(self):
        
        driver = self.browser
        rootUrl = self.mainRootUrl
        
        driver.get(rootUrl)
        
        contacts = driver.find_elements_by_css_selector(".contactperson-detail-item .apo-xs-col-6 .apo-md-col-2")
        name = contacts.find_elements_by_css_selector(".name").text
        # print(name)
        
        return contacts

In [ ]:
# cDriver = chromeDriver()

In [ ]:
name_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-detail-list')]/article[contains(@class, 'contactperson-detail-item')]/span[contains(@class, 'name')]").text
name_full_list = name_raw.split(" ")
try:
    if len(name_full_list) > 2 and name_full_list[2] != "":
        name = ' '.join(name_full_list[:len(name_full_list) - 1])
        surname = name_full_list[-1:]
    else:
        name = name_full_list[0]
        surname = name_full_list[1]
except:
    name = ""
    surname = ""
    
print(name)
print(surname)

In [ ]:
position_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-detail-list')]/article[contains(@class, 'contactperson-detail-item')]").text
position = position_raw.split("\n")[1]
qualifikation = position_raw.split("\n")[2]
print(position)
print(qualifikation)

In [ ]:
mobile_phonenumber_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-detail-list')]/article[contains(@class, 'contactperson-detail-item')]/a[contains(@href, 'Mobil')]").text
mobile_phonenumber = mobile_phonenumber_raw.replace("Mobil: ","").replace(" ","").replace("/","").replace("-","")
telefon_phonenumber_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-detail-list')]/article[contains(@class, 'contactperson-detail-item')]/a[contains(@href, 'Mobil')]").text
telefon_phonenumber = telefon_phonenumber_raw.replace("Mobil: ","").replace(" ","").replace("/","").replace("-","")

if "0" in mobile_phonenumber:
    phonenumber = mobile_phonenumber
else:
    phonenumber = telefon_phonenumber_raw

phonenumber

In [ ]:
image = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-detail-list')]/article[contains(@class, 'contactperson-detail-item')]/div/img").get_attribute("src")
image

In [ ]:
address_raw = driver.find_element_by_xpath("//div[contains(@class, 'contactperson-location') and contains(@class, 'js-contactperson-location')]/article[contains(@class, 'contactperson-overview-item')]/p").text
address_list = address_raw.split("\n")
street = address_list[0]
city_list_raw = address_list[1].split(" ")
plz = city_list_raw[0]
city = " ".join(city_list_raw[1:])
print(street)
print(plz)
print(city)

In [ ]:
organisation = "ApoFinanz"

In [ ]:
Employee(surname=surname, name=name, image=image, phonenumber=phonenumber, position=position, city=city, plz=plz, street=street, organisation=organisation, qualifikation=qualifikation).add_employee()

In [ ]:
Employee.get_employees()

In [ ]:
Employee(surname=surname, name=name, image=image, phonenumber=phonenumber, position=position, city=city, plz=plz, street=street, organisation=organisation).add_employee()